# FingerMovements

In [7]:
###libraries

#Data manipulation
import pandas as pd
import numpy as np
from collections import Counter
from sklearn import preprocessing
import sys
import os

#Time series transformers
from pyts.multivariate.classification import MultivariateClassifier
from pyts.multivariate.transformation import WEASELMUSE
from pyts.classification import KNeighborsClassifier
from pywt import wavedec
import pyeeg
import scipy.stats

#Classifiers
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier

#Deep Learning
import mcfly

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from keras import backend
from numba import cuda 

#Random
import multiprocessing
from multiprocessing import Pool
import gc
from tqdm.notebook import tqdm

np.random.seed(42)

In [8]:
###Read the data
dataset = 'FingerMovements'

X_train = np.load('Datasets_clean/{}/X_train.npy'.format(dataset))
y_train = np.load('Datasets_clean/{}/y_train.npy'.format(dataset))
X_test = np.load('Datasets_clean/{}/X_test.npy'.format(dataset))
y_test = np.load('Datasets_clean/{}/y_test.npy'.format(dataset))

X_full = np.vstack([X_train,X_test])
y_full = np.hstack([y_train,y_test])

#Create scores dict
scores = {}

### Functions

In [9]:
def ResampleLinear1D(original, targetLen = 40):
    original = np.array(original, dtype=np.float)
    index_arr = np.linspace(0, len(original)-1, num=targetLen, dtype=np.float)
    index_floor = np.array(index_arr, dtype=np.int) #Round down
    index_ceil = index_floor + 1
    index_rem = index_arr - index_floor #Remain

    val1 = original[index_floor]
    val2 = original[index_ceil % len(original)]
    interp = val1 * (1.0-index_rem) + val2 * index_rem
    assert(len(interp) == targetLen)
    return interp

def calculate_entropy(list_values):
    counter_values = Counter(list_values).most_common()
    probabilities = [elem[1]/len(list_values) for elem in counter_values]
    entropy=scipy.stats.entropy(probabilities)
    return entropy
 
def calculate_statistics(list_values):
    n5 = np.nanpercentile(list_values, 5)
    n25 = np.nanpercentile(list_values, 25)
    n75 = np.nanpercentile(list_values, 75)
    n95 = np.nanpercentile(list_values, 95)
    median = np.nanpercentile(list_values, 50)
    mean = np.nanmean(list_values)
    std = np.nanstd(list_values)
    var = np.nanvar(list_values)
    rms = np.nanmean(np.sqrt(list_values**2))
    return [n5, n25, n75, n95, median, mean, std, var, rms]
 
def calculate_crossings(list_values):
    zero_crossing_indices = np.nonzero(np.diff(np.array(list_values) > 0))[0]
    no_zero_crossings = len(zero_crossing_indices)
    mean_crossing_indices = np.nonzero(np.diff(np.array(list_values) > np.nanmean(list_values)))[0]
    no_mean_crossings = len(mean_crossing_indices)
    return [no_zero_crossings, no_mean_crossings]
 
def get_features(list_values):
    entropy = calculate_entropy(list_values)
    crossings = calculate_crossings(list_values)
    statistics = calculate_statistics(list_values)
    return [entropy] + crossings + statistics

### DTW + 1-Knn

In [10]:
dtw_acc = []

skf = StratifiedKFold(n_splits=10)
for train_index, test_index in tqdm(skf.split(X_full, y_full)):
    
    X_train = X_full[train_index]
    y_train = y_full[train_index]
    X_test = X_full[test_index]
    y_test = y_full[test_index]
    
#     X_train = np.apply_along_axis(ResampleLinear1D, axis = 2, arr = X_train)
#     X_test = np.apply_along_axis(ResampleLinear1D, axis = 2, arr = X_test)
    
    dtw_knn = MultivariateClassifier(KNeighborsClassifier(metric = 'dtw_itakura',
                                                      n_jobs = -1,
                                                      metric_params = {'max_slope' : 2}))
    
    dtw_knn.fit(X_train,y_train)
    acc = dtw_knn.score(X_test,y_test)
    dtw_acc.append(acc)
    
scores['dtw_knn'] = [np.mean(dtw_acc), np.std(dtw_acc)]

### WEASELMUSE

In [38]:
accuracies_lr = []
accuracies_rf = []
accuracies_svc = []
accuracies_lgbm = []

skf = StratifiedKFold(n_splits=10)
for train_index, test_index in tqdm(skf.split(X_full, y_full)):
    
    X_train = X_full[train_index]
    y_train = y_full[train_index]
    X_test = X_full[test_index]
    y_test = y_full[test_index]

    #Optimal hyperparameters
    hyperparameters = {}

    #wm + lr    
    hyperparameters['wm_lr'] = {}
    hyperparameters['wm_lr']['word_size'] = 2
    hyperparameters['wm_lr']['n_bins'] = 3
    hyperparameters['wm_lr']['C'] = 0.5

    wm = WEASELMUSE(word_size = hyperparameters['wm_lr']['word_size'], 
                    n_bins = hyperparameters['wm_lr']['n_bins'])

    logistic = LogisticRegression(solver = 'lbfgs',
                                  multi_class = 'auto',
                                  max_iter=3000,
                                  C = hyperparameters['wm_lr']['C'])

    clf = make_pipeline(wm, logistic)
    clf.fit(X_train,y_train)
    acc = clf.score(X_test,y_test)
    accuracies_lr.append(np.round(acc,2))

    #wm + rf    
    hyperparameters['wm_rf'] = {}
    hyperparameters['wm_rf']['word_size'] = 2
    hyperparameters['wm_rf']['n_bins'] = 3
    hyperparameters['wm_rf']['n_estimators'] = 300
    hyperparameters['wm_rf']['max_depth'] = 9

    wm = WEASELMUSE(word_size = hyperparameters['wm_rf']['word_size'], 
                    n_bins = hyperparameters['wm_rf']['n_bins'])

    rf = RandomForestClassifier(n_estimators = hyperparameters['wm_rf']['n_estimators'],
                                max_depth = hyperparameters['wm_rf']['max_depth'])

    clf = make_pipeline(wm, rf)
    clf.fit(X_train,y_train)
    acc = clf.score(X_test,y_test)
    accuracies_rf.append(np.round(acc,2))
    
    #wm + svc
    hyperparameters['wm_svc'] = {}
    hyperparameters['wm_svc']['word_size'] = 2
    hyperparameters['wm_svc']['n_bins'] = 3
    hyperparameters['wm_svc']['C'] = 1
    hyperparameters['wm_svc']['kernel'] = 'rbf'
    hyperparameters['wm_svc']['degree'] = 3
    hyperparameters['wm_svc']['gamma'] = 'scale'

    wm = WEASELMUSE(word_size = hyperparameters['wm_svc']['word_size'], 
                    n_bins = hyperparameters['wm_svc']['n_bins'])

    svc = SVC(C = hyperparameters['wm_svc']['C'],
              kernel = hyperparameters['wm_svc']['kernel'],
              degree = hyperparameters['wm_svc']['degree'],
              gamma = hyperparameters['wm_svc']['gamma'])

    clf = make_pipeline(wm, svc)
    clf.fit(X_train,y_train)
    acc = clf.score(X_test,y_test)
    accuracies_svc.append(np.round(acc,2))

    #wm + lgbm
    hyperparameters['wm_lgbm'] = {}
    hyperparameters['wm_lgbm']['word_size'] = 4
    hyperparameters['wm_lgbm']['n_bins'] = 5
    hyperparameters['wm_lgbm']['num_leaves'] = 92
    hyperparameters['wm_lgbm']['max_depth'] = 3
    hyperparameters['wm_lgbm']['learning_rate'] = 0.3
    hyperparameters['wm_lgbm']['n_estimators'] = 250
    hyperparameters['wm_lgbm']['min_split_gain'] = 0.2
    hyperparameters['wm_lgbm']['min_child_samples'] = 20
    hyperparameters['wm_lgbm']['colsample_bytree'] = 1
    hyperparameters['wm_lgbm']['reg_alpha'] = 0.3
    hyperparameters['wm_lgbm']['reg_lambda'] = 0.2

    wm = WEASELMUSE(word_size = hyperparameters['wm_svc']['word_size'], 
                    n_bins = hyperparameters['wm_svc']['n_bins'])

    def sparse_float(mat):
        return mat.astype('float')

    trans_sparse_float = FunctionTransformer(sparse_float, validate = False)

    lgbm = LGBMClassifier(n_jobs = -1,
                          num_leaves = hyperparameters['wm_lgbm']['num_leaves'],
                          max_depth = hyperparameters['wm_lgbm']['max_depth'],
                          learning_rate = hyperparameters['wm_lgbm']['learning_rate'],
                          n_estimators = hyperparameters['wm_lgbm']['n_estimators'],
                          min_split_gain = hyperparameters['wm_lgbm']['min_split_gain'],
                          min_child_samples = hyperparameters['wm_lgbm']['min_child_samples'],
                          colsample_by_tree = hyperparameters['wm_lgbm']['colsample_bytree'],
                          reg_alpha = hyperparameters['wm_lgbm']['reg_alpha'],
                          reg_lambda = hyperparameters['wm_lgbm']['reg_lambda'])

    clf = make_pipeline(wm, trans_sparse_float, lgbm)
    clf.fit(X_train,y_train)
    acc = clf.score(X_test,y_test)
    accuracies_lgbm.append(np.round(acc,2))
    
scores['wm_lr'] = [np.mean(accuracies_lr), np.std(accuracies_lr)]
scores['wm_rf'] = [np.mean(accuracies_rf), np.std(accuracies_rf)]
scores['wm_svc'] = [np.mean(accuracies_svc), np.std(accuracies_svc)]
scores['wm_lgbm'] = [np.mean(accuracies_lgbm), np.std(accuracies_lgbm)]

### Deep Learning

In [12]:
def dl_func(dl_type,
            X_train_dl, 
            y_train_dl,
            X_val_dl,
            y_val_dl,
            X_test_dl,
            y_test_dl):
    
    #Validate diferent architectures
    num_of_candidate_models = 8
    random_search_epoches = 100
    random_search_es = 30
    best_model_epoches = 200
    best_model_es = 30

    for mod_type in [dl_type]:

        #Create architectures
        num_classes = y_train_dl.shape[1]
        metric = 'accuracy'
        models = mcfly.modelgen.generate_models(X_train_dl.shape,
                                                number_of_classes=num_classes,
                                                number_of_models = num_of_candidate_models,
                                                model_types = [mod_type],
                                                metrics=[metric])

        #Save intermediate results
        resultpath = 'temp'
        outputfile = os.path.join(resultpath, 'modelcomparison_{}_{}.json'.format(mod_type,dataset))

        #Find best architecture
        histories, val_accuracies, val_losses = mcfly.find_architecture.train_models_on_samples(X_train_dl, y_train_dl,
                                                                                                X_val_dl, y_val_dl,
                                                                                                models,
                                                                                                nr_epochs=random_search_epoches,
                                                                                                subset_size=None,
                                                                                                verbose=False,
                                                                                                outputfile=outputfile,
                                                                                                metric=metric,
                                                                                                early_stopping_patience=random_search_es
                                                                                                )

        #Select and train best architecture
        best_model_index = np.argmax(val_accuracies)
        best_model, best_params, best_model_types = models[best_model_index]

        es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=best_model_es)
        mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

        history = best_model.fit(X_train_dl, y_train_dl,
                                 epochs=best_model_epoches, validation_data=(X_val_dl, y_val_dl),
                                 callbacks = [es,mc])

        #Accuracy in test dataset
        saved_model = load_model('best_model.h5')
        return saved_model.evaluate(X_test_dl, y_test_dl, verbose = False)[1]

In [ ]:
accuracies_cnn = []
accuracies_it = []

skf = StratifiedKFold(n_splits=10)
for train_index, test_index in tqdm(skf.split(X_full, y_full)):
    
    X_train = X_full[train_index]
    y_train = y_full[train_index]
    X_test = X_full[test_index]
    y_test = y_full[test_index]

    #Create train, validation and test sets for DL
    LE = LabelEncoder()
    skf_train_val = StratifiedKFold(n_splits=5)

    for train_index, val_index in skf_train_val.split(X_train, y_train):
        train_index = train_index
        val_index = val_index

    X_train_val = X_train.copy()

    X_train_dl = X_train_val[train_index].copy()
    X_train_dl = np.transpose(X_train_dl, (0,2,1))
    y_train_dl = y_train[train_index].copy()
    y_train_dl = to_categorical(LE.fit_transform(y_train_dl))

    X_val_dl = X_train_val[val_index].copy()
    X_val_dl = np.transpose(X_val_dl, (0,2,1))
    y_val_dl = y_train[val_index].copy()
    y_val_dl = to_categorical(LE.transform(y_val_dl))

    X_test_dl = np.transpose(X_test, (0,2,1))
    y_test_dl = to_categorical(LE.transform(y_test))
    
    acc_cnn = dl_func('CNN',
                     X_train_dl, 
                     y_train_dl,
                     X_val_dl,
                     y_val_dl,
                     X_test_dl,
                     y_test_dl)
    
    acc_it = dl_func('InceptionTime',
                     X_train_dl, 
                     y_train_dl,
                     X_val_dl,
                     y_val_dl,
                     X_test_dl,
                     y_test_dl)

    accuracies_cnn.append(acc_cnn.copy())
    accuracies_it.append(acc_it.copy())

scores['dl_cnn'] = [np.mean(accuracies_cnn), np.std(accuracies_cnn)]
scores['dl_it'] = [np.mean(accuracies_it), np.std(accuracies_it)]

The value of model_types is set from ['CNN', 'DeepConvLSTM', 'ResNet', 'InceptionTime'] (default) to ['CNN']
The value of metrics is set from ['accuracy'] (default) to ['accuracy']
Set maximum kernel size for InceptionTime models to number of timesteps.
Train on 300 samples, validate on 74 samples
Epoch 1/200
224/300 [=====================>........] - ETA: 0s - loss: 0.1826 - accuracy: 0.9955
Epoch 00001: val_accuracy improved from -inf to 0.56757, saving model to best_model.h5
300/300 [==============================] - 0s 1ms/sample - loss: 0.1804 - accuracy: 0.9967 - val_loss: 0.8881 - val_accuracy: 0.5676
Epoch 2/200
224/300 [=====================>........] - ETA: 0s - loss: 0.1735 - accuracy: 1.0000
Epoch 00002: val_accuracy improved from 0.56757 to 0.59459, saving model to best_model.h5
300/300 [==============================] - 0s 1ms/sample - loss: 0.1750 - accuracy: 1.0000 - val_loss: 0.9009 - val_accuracy: 0.5946
Epoch 3/200
288/300 [===========================>..] - ETA: 0s -

Epoch 28/200
224/300 [=====================>........] - ETA: 0s - loss: 0.1504 - accuracy: 1.0000
Epoch 00028: val_accuracy did not improve from 0.62162
300/300 [==============================] - 0s 821us/sample - loss: 0.1504 - accuracy: 1.0000 - val_loss: 0.8718 - val_accuracy: 0.5676
Epoch 29/200
224/300 [=====================>........] - ETA: 0s - loss: 0.1556 - accuracy: 1.0000
Epoch 00029: val_accuracy did not improve from 0.62162
300/300 [==============================] - 0s 808us/sample - loss: 0.1565 - accuracy: 1.0000 - val_loss: 0.8741 - val_accuracy: 0.5811
Epoch 30/200
224/300 [=====================>........] - ETA: 0s - loss: 0.1496 - accuracy: 1.0000
Epoch 00030: val_accuracy did not improve from 0.62162
300/300 [==============================] - 0s 824us/sample - loss: 0.1512 - accuracy: 1.0000 - val_loss: 0.8544 - val_accuracy: 0.5676
Epoch 31/200
224/300 [=====================>........] - ETA: 0s - loss: 0.1539 - accuracy: 1.0000
Epoch 00031: val_accuracy did not impr

288/300 [===========================>..] - ETA: 0s - loss: 0.1126 - accuracy: 0.9653
Epoch 00008: val_accuracy did not improve from 0.62162
300/300 [==============================] - 0s 1ms/sample - loss: 0.1101 - accuracy: 0.9667 - val_loss: 1.5907 - val_accuracy: 0.6216
Epoch 9/200
288/300 [===========================>..] - ETA: 0s - loss: 0.0981 - accuracy: 0.9653
Epoch 00009: val_accuracy did not improve from 0.62162
300/300 [==============================] - 0s 1ms/sample - loss: 0.1030 - accuracy: 0.9633 - val_loss: 1.8490 - val_accuracy: 0.6081
Epoch 10/200
288/300 [===========================>..] - ETA: 0s - loss: 0.0642 - accuracy: 0.9826
Epoch 00010: val_accuracy did not improve from 0.62162
300/300 [==============================] - 0s 1ms/sample - loss: 0.0682 - accuracy: 0.9800 - val_loss: 1.3999 - val_accuracy: 0.5946
Epoch 11/200
288/300 [===========================>..] - ETA: 0s - loss: 0.0642 - accuracy: 0.9826
Epoch 00011: val_accuracy did not improve from 0.62162
300

224/300 [=====================>........] - ETA: 0s - loss: 0.1832 - accuracy: 1.0000
Epoch 00032: val_accuracy did not improve from 0.63514
300/300 [==============================] - 0s 854us/sample - loss: 0.1877 - accuracy: 1.0000 - val_loss: 1.0159 - val_accuracy: 0.5676
Epoch 33/200
224/300 [=====================>........] - ETA: 0s - loss: 0.1863 - accuracy: 1.0000
Epoch 00033: val_accuracy did not improve from 0.63514
300/300 [==============================] - 0s 845us/sample - loss: 0.1856 - accuracy: 1.0000 - val_loss: 1.0093 - val_accuracy: 0.5676
Epoch 34/200
288/300 [===========================>..] - ETA: 0s - loss: 0.1884 - accuracy: 1.0000
Epoch 00034: val_accuracy did not improve from 0.63514
300/300 [==============================] - 0s 855us/sample - loss: 0.1881 - accuracy: 1.0000 - val_loss: 1.0102 - val_accuracy: 0.5405
Epoch 35/200
224/300 [=====================>........] - ETA: 0s - loss: 0.1779 - accuracy: 1.0000
Epoch 00035: val_accuracy did not improve from 0.63

300/300 [==============================] - 0s 1ms/sample - loss: 0.3354 - accuracy: 0.9767 - val_loss: 1.4736 - val_accuracy: 0.5541
Epoch 2/200
288/300 [===========================>..] - ETA: 0s - loss: 0.3233 - accuracy: 0.9861
Epoch 00002: val_accuracy improved from 0.55405 to 0.62162, saving model to best_model.h5
300/300 [==============================] - 0s 1ms/sample - loss: 0.3273 - accuracy: 0.9833 - val_loss: 1.3292 - val_accuracy: 0.6216
Epoch 3/200
288/300 [===========================>..] - ETA: 0s - loss: 0.3138 - accuracy: 0.9861
Epoch 00003: val_accuracy did not improve from 0.62162
300/300 [==============================] - 0s 925us/sample - loss: 0.3142 - accuracy: 0.9867 - val_loss: 1.3663 - val_accuracy: 0.5000
Epoch 4/200
288/300 [===========================>..] - ETA: 0s - loss: 0.3083 - accuracy: 0.9826
Epoch 00004: val_accuracy did not improve from 0.62162
300/300 [==============================] - 0s 950us/sample - loss: 0.3071 - accuracy: 0.9833 - val_loss: 1.3

...

Epoch 69/200
288/300 [===========================>..] - ETA: 0s - loss: 0.6035 - accuracy: 0.6771
Epoch 00069: val_accuracy did not improve from 0.70270
300/300 [==============================] - 0s 991us/sample - loss: 0.6010 - accuracy: 0.6800 - val_loss: 0.6391 - val_accuracy: 0.5676
Epoch 70/200
288/300 [===========================>..] - ETA: 0s - loss: 0.5980 - accuracy: 0.6701
Epoch 00070: val_accuracy did not improve from 0.70270
300/300 [==============================] - 0s 1ms/sample - loss: 0.6036 - accuracy: 0.6667 - val_loss: 0.6194 - val_accuracy: 0.6081
Epoch 71/200
288/300 [===========================>..] - ETA: 0s - loss: 0.5925 - accuracy: 0.7049
Epoch 00071: val_accuracy did not improve from 0.70270
300/300 [==============================] - 0s 990us/sample - loss: 0.5934 - accuracy: 0.7067 - val_loss: 0.6540 - val_accuracy: 0.6351
Epoch 72/200
288/300 [===========================>..] - ETA: 0s - loss: 0.6260 - accuracy: 0.6701
Epoch 00072: val_accuracy did not improv

Epoch 10/200
288/300 [===========================>..] - ETA: 0s - loss: 0.5803 - accuracy: 0.7431
Epoch 00010: val_accuracy did not improve from 0.60811
300/300 [==============================] - 0s 1ms/sample - loss: 0.5968 - accuracy: 0.7400 - val_loss: 0.7785 - val_accuracy: 0.5135
Epoch 11/200
288/300 [===========================>..] - ETA: 0s - loss: 0.5158 - accuracy: 0.7986
Epoch 00011: val_accuracy did not improve from 0.60811
300/300 [==============================] - 0s 1ms/sample - loss: 0.5114 - accuracy: 0.8067 - val_loss: 0.7547 - val_accuracy: 0.5541
Epoch 12/200
288/300 [===========================>..] - ETA: 0s - loss: 0.4870 - accuracy: 0.8576
Epoch 00012: val_accuracy did not improve from 0.60811
300/300 [==============================] - 0s 1ms/sample - loss: 0.4850 - accuracy: 0.8600 - val_loss: 0.8087 - val_accuracy: 0.5676
Epoch 13/200
288/300 [===========================>..] - ETA: 0s - loss: 0.4686 - accuracy: 0.8264
Epoch 00013: val_accuracy did not improve fr

Epoch 39/200
288/300 [===========================>..] - ETA: 0s - loss: 0.4674 - accuracy: 0.8333
Epoch 00039: val_accuracy did not improve from 0.63514
300/300 [==============================] - 0s 1ms/sample - loss: 0.4676 - accuracy: 0.8367 - val_loss: 0.8859 - val_accuracy: 0.5405
Epoch 40/200
288/300 [===========================>..] - ETA: 0s - loss: 0.4680 - accuracy: 0.8194
Epoch 00040: val_accuracy did not improve from 0.63514
300/300 [==============================] - 0s 1ms/sample - loss: 0.4686 - accuracy: 0.8200 - val_loss: 0.8711 - val_accuracy: 0.5405
Epoch 41/200
288/300 [===========================>..] - ETA: 0s - loss: 0.4720 - accuracy: 0.8299
Epoch 00041: val_accuracy did not improve from 0.63514
300/300 [==============================] - 0s 1ms/sample - loss: 0.4815 - accuracy: 0.8233 - val_loss: 0.8932 - val_accuracy: 0.5135
Epoch 42/200
288/300 [===========================>..] - ETA: 0s - loss: 0.6059 - accuracy: 0.7604
Epoch 00042: val_accuracy did not improve fr

224/300 [=====================>........] - ETA: 0s - loss: 0.1423 - accuracy: 1.0000
Epoch 00006: val_accuracy did not improve from 0.64865
300/300 [==============================] - 0s 815us/sample - loss: 0.1684 - accuracy: 0.9900 - val_loss: 0.9670 - val_accuracy: 0.5676
Epoch 7/200
224/300 [=====================>........] - ETA: 0s - loss: 0.1673 - accuracy: 0.9866
Epoch 00007: val_accuracy did not improve from 0.64865
300/300 [==============================] - 0s 814us/sample - loss: 0.1678 - accuracy: 0.9867 - val_loss: 1.0548 - val_accuracy: 0.5811
Epoch 8/200
224/300 [=====================>........] - ETA: 0s - loss: 0.1557 - accuracy: 1.0000
Epoch 00008: val_accuracy did not improve from 0.64865
300/300 [==============================] - 0s 809us/sample - loss: 0.1535 - accuracy: 1.0000 - val_loss: 1.0278 - val_accuracy: 0.6351
Epoch 9/200
224/300 [=====================>........] - ETA: 0s - loss: 0.1499 - accuracy: 1.0000
Epoch 00009: val_accuracy did not improve from 0.64865

288/300 [===========================>..] - ETA: 0s - loss: 0.0424 - accuracy: 0.9896
Epoch 00012: val_accuracy did not improve from 0.56757
300/300 [==============================] - 0s 1ms/sample - loss: 0.0429 - accuracy: 0.9900 - val_loss: 1.7763 - val_accuracy: 0.5135
Epoch 13/200
288/300 [===========================>..] - ETA: 0s - loss: 0.0505 - accuracy: 0.9792
Epoch 00013: val_accuracy did not improve from 0.56757
300/300 [==============================] - 0s 984us/sample - loss: 0.0532 - accuracy: 0.9767 - val_loss: 1.6738 - val_accuracy: 0.5270
Epoch 14/200
288/300 [===========================>..] - ETA: 0s - loss: 0.0505 - accuracy: 0.9826
Epoch 00014: val_accuracy did not improve from 0.56757
300/300 [==============================] - 0s 1ms/sample - loss: 0.0565 - accuracy: 0.9800 - val_loss: 1.7005 - val_accuracy: 0.5135
Epoch 15/200
288/300 [===========================>..] - ETA: 0s - loss: 0.0487 - accuracy: 0.9861
Epoch 00015: val_accuracy improved from 0.56757 to 0.59

### PyEEG

In [40]:
pyeeg_lr_acc = []
pyeeg_rf_acc = []
pyeeg_svc_acc = []
pyeeg_lgbm_acc = []

skf = StratifiedKFold(n_splits=10)
for train_index, test_index in tqdm(skf.split(X_full, y_full)):
    
    X_train = X_full[train_index]
    y_train = y_full[train_index]
    X_test = X_full[test_index]
    y_test = y_full[test_index]
    
    feat_dict = {}

    for df,name in zip([X_train,X_test],['X_train_pyeeg','X_test_pyeeg']):
        feat_dict[name] = np.array([]).reshape(-1,2100)

        for ind in range(0,df.shape[0]):
            features = np.array([])

            for channel in range(0,df[ind].shape[0]):

                eeg_series = df[ind][channel]

                coefs = wavedec(eeg_series, 'db4', level=4)

                for band in coefs:
                    features = np.hstack([features,get_features(band)])

                fft_power = pyeeg.bin_power(eeg_series, Band = [0.5,4,7,12,30,100], Fs = 256)
                power_spectral_intensity = fft_power[0]
                relative_intensity_ratio = fft_power[1]
#                 fisher_info = pyeeg.fisher_info(eeg_series, Tau = 1, DE = 90)
                pfd = pyeeg.pfd(eeg_series)
                spectral_entropy = pyeeg.spectral_entropy(eeg_series, Band = [0.5,4,7,12,30,100], Fs = 256,
                                                          Power_Ratio = relative_intensity_ratio)

#                 eeg_series = ResampleLinear1D(eeg_series,200)

#                 ap_entropy = pyeeg.ap_entropy(eeg_series, M = 3, R = 0.25*np.std(eeg_series))
                hfd = pyeeg.hfd(eeg_series, Kmax = 10)
                hjorth = pyeeg.hjorth(eeg_series)
                hjorth_mob = hjorth[0]
                hjorth_comp = hjorth[1]
    #             hurst = pyeeg.hurst(eeg_series)


                features = np.hstack([features, power_spectral_intensity, relative_intensity_ratio,
                                      hfd, hjorth_mob, hjorth_comp, pfd, spectral_entropy])


            feat_dict[name] = np.vstack([feat_dict[name],features])

    X_train_pyeeg = feat_dict['X_train_pyeeg'].copy()
    X_test_pyeeg = feat_dict['X_test_pyeeg'].copy()
    
    X_train_pyeeg = np.nan_to_num(X_train_pyeeg)
    X_test_pyeeg = np.nan_to_num(X_test_pyeeg)
    scaler = preprocessing.StandardScaler().fit(X_train_pyeeg)
    X_train_pyeeg = scaler.transform(X_train_pyeeg)
    X_test_pyeeg = scaler.transform(X_test_pyeeg)
    
    #pyeeg + lr    
    hyperparameters = {}
    hyperparameters['pyeeg_lr'] = {}
    hyperparameters['pyeeg_lr']['C'] = 1

    logistic = LogisticRegression(solver = 'lbfgs',
                                  multi_class = 'auto',
                                  max_iter=3000,
                                  C = hyperparameters['pyeeg_lr']['C'])

    logistic.fit(X_train_pyeeg,y_train)
    acc = logistic.score(X_test_pyeeg,y_test)
    pyeeg_lr_acc.append(np.round(acc,2).copy())
    
    #pyeeg + rf    
    hyperparameters = {}
    hyperparameters['pyeeg_rf'] = {}
    hyperparameters['pyeeg_rf']['n_estimators'] = 500
    hyperparameters['pyeeg_rf']['max_depth'] = 3

    rf = RandomForestClassifier(n_estimators = hyperparameters['pyeeg_rf']['n_estimators'],
                                max_depth = hyperparameters['pyeeg_rf']['max_depth'])

    rf.fit(X_train_pyeeg,y_train)
    acc = rf.score(X_test_pyeeg,y_test)
    pyeeg_rf_acc.append(np.round(acc,2).copy())
    
    #pyeeg + svc
    hyperparameters = {}
    hyperparameters['pyeeg_svc'] = {}
    hyperparameters['pyeeg_svc']['C'] = 2
    hyperparameters['pyeeg_svc']['kernel'] = 'rbf'
    hyperparameters['pyeeg_svc']['degree'] = 3
    hyperparameters['pyeeg_svc']['gamma'] = 'scale'

    svc = SVC(C = hyperparameters['pyeeg_svc']['C'],
              kernel = hyperparameters['pyeeg_svc']['kernel'],
              degree = hyperparameters['pyeeg_svc']['degree'],
              gamma = hyperparameters['pyeeg_svc']['gamma'])

    clf = make_pipeline(svc)
    clf.fit(X_train_pyeeg,y_train)
    acc = clf.score(X_test_pyeeg,y_test)
    pyeeg_svc_acc.append(np.round(acc,2).copy())

    
    #pyeeg + lgbm
    hyperparameters = {}
    hyperparameters['pyeeg_lgbm'] = {}
    hyperparameters['pyeeg_lgbm']['num_leaves'] = 254
    hyperparameters['pyeeg_lgbm']['max_depth'] = 2
    hyperparameters['pyeeg_lgbm']['learning_rate'] = 0.1
    hyperparameters['pyeeg_lgbm']['n_estimators'] = 500
    hyperparameters['pyeeg_lgbm']['min_split_gain'] = 0.2
    hyperparameters['pyeeg_lgbm']['min_child_samples'] = 20
    hyperparameters['pyeeg_lgbm']['colsample_bytree'] = 1
    hyperparameters['pyeeg_lgbm']['reg_alpha'] = 0.1
    hyperparameters['pyeeg_lgbm']['reg_lambda'] = 0

    def sparse_float(mat):
        return mat.astype('float')

    trans_sparse_float = FunctionTransformer(sparse_float, validate = False)

    lgbm = LGBMClassifier(n_jobs = -1,
                          num_leaves = hyperparameters['pyeeg_lgbm']['num_leaves'],
                          max_depth = hyperparameters['pyeeg_lgbm']['max_depth'],
                          learning_rate = hyperparameters['pyeeg_lgbm']['learning_rate'],
                          n_estimators = hyperparameters['pyeeg_lgbm']['n_estimators'],
                          min_split_gain = hyperparameters['pyeeg_lgbm']['min_split_gain'],
                          min_child_samples = hyperparameters['pyeeg_lgbm']['min_child_samples'],
                          colsample_by_tree = hyperparameters['pyeeg_lgbm']['colsample_bytree'],
                          reg_alpha = hyperparameters['pyeeg_lgbm']['reg_alpha'],
                          reg_lambda = hyperparameters['pyeeg_lgbm']['reg_lambda'])

    clf = make_pipeline(trans_sparse_float, lgbm)
    clf.fit(X_train_pyeeg,y_train)
    acc = clf.score(X_test_pyeeg,y_test)
    pyeeg_lgbm_acc.append(np.round(acc,2).copy())

scores['pyeeg_lr'] = [np.mean(pyeeg_lr_acc), np.std(pyeeg_lr_acc)]
scores['pyeeg_rf'] = [np.mean(pyeeg_rf_acc), np.std(pyeeg_rf_acc)]
scores['pyeeg_svc'] = [np.mean(pyeeg_svc_acc), np.std(pyeeg_svc_acc)]
scores['pyeeg_lgbm'] = [np.mean(pyeeg_lgbm_acc), np.std(pyeeg_lgbm_acc)]


/jet/prs/workspace/tf_2/lib/python3.6/site-packages/pywt/_multilevel.py:45: UserWarning: Level value of 4 is too high: all coefficients will experience boundary effects.
  "boundary effects.").format(level))
/jet/prs/workspace/tf_2/lib/python3.6/site-packages/pyeeg-0.4.4-py3.6.egg/pyeeg/entropy.py:235: RuntimeWarning: divide by zero encountered in log
/jet/prs/workspace/tf_2/lib/python3.6/site-packages/pyeeg-0.4.4-py3.6.egg/pyeeg/entropy.py:235: RuntimeWarning: invalid value encountered in double_scalars
/jet/prs/workspace/tf_2/lib/python3.6/site-packages/pyeeg-0.4.4-py3.6.egg/pyeeg/fractal_dimension.py:22: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
/jet/prs/workspace/tf_2/lib/python3.6/site-packages/pywt/_multilevel.py:45: UserWarning: Level value of

In [41]:
#Salva scores
individual_results = pd.DataFrame({'dtw_acc' : dtw_acc,
                                   'wm_lr_acc' : accuracies_lr,
                                   'wm_rf_acc' : accuracies_rf,
                                   'wm_svc_acc' : accuracies_svc,
                                   'wm_lgbm_acc' : accuracies_lgbm,
                                   'cnn_acc' : accuracies_cnn,
                                   'it_acc' : accuracies_it,
                                   'pyeeg_lr_acc' : pyeeg_lr_acc,
                                   'pyeeg_rf_acc' : pyeeg_rf_acc,
                                   'pyeeg_svc_acc' : pyeeg_svc_acc,
                                   'pyeeg_lgbm_acc' : pyeeg_lgbm_acc})

individual_results.to_csv('results/Individual_Scores_{}.csv'.format(dataset), index = False)

results = pd.DataFrame(scores).T.reset_index().rename(columns={0 : 'mean',
                                                               1 : 'std',
                                                               'index' : 'method'}).sort_values('mean', ascending = False)

results.to_csv('results/Scores_{}.csv'.format(dataset), index = False)

In [42]:
results

,method,mean,std
9,pyeeg_svc,0.588000,0.058275
8,pyeeg_rf,0.581000,0.063632
10,pyeeg_lgbm,0.556000,0.089688
6,dl_it,0.555459,0.046151
7,pyeeg_lr,0.548000,0.058103
0,dtw_knn,0.540883,0.061287
2,wm_rf,0.535000,0.088459
1,wm_lr,0.524000,0.051029
5,dl_cnn,0.521777,0.061977
4,wm_lgbm,0.521000,0.073410
